In [1]:
import serpapi
import pandas as pd

In [2]:
MARKETS = [
    'br'
]

APP_ID = '567763947'

In [ ]:
client = serpapi.Client(api_key='INSERT API KEY HERE')

In [4]:
from datetime import datetime

import re

# due to a lack of date string pattern and\or fragmentation,
# the filter will be made by regex
def filter_2024(review):
    pattern = re.compile(r'2024')
    pattern_2025 = re.compile(r'2025')
    return bool(pattern.search(review['review_date'])) or bool(pattern_2025.search(review['review_date']))

def fetch_reviews_2024(id, country):
    page = 1
    reviews = []
    since = datetime.strptime('2024-01-01', '%Y-%m-%d')
    try:
        while True:
            print(f'Fetching reviews for {id} ({country})...')
            params = {
                'product_id': id,
                'engine': 'apple_reviews',
                'sort': 'mostrecent',
                'country': country,
                'page': page
            }
            result = client.search(params=params)
            if result.get('error'):
                print(f'Error: {result['error']}')
                break
            since_reviews = list(filter(
                filter_2024, result['reviews']
            ))
            if len(since_reviews) == 0: break
            page += 1
            reviews += since_reviews
        print(f'Reviews fetched: {len(reviews)}')
        return reviews
    except Exception as e:
        print(f'Error fetching reviews: {e}')
        return reviews

In [5]:
market_reviews = {}

for market in MARKETS:
    reviews = fetch_reviews_2024(APP_ID, market)
    market_reviews[market] = reviews

Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 567763947 (br)...
Fetching reviews for 5677

In [6]:
for market in market_reviews:
    print(f'Reviews fetched for {market}: {len(market_reviews[market])}')

Reviews fetched for br: 1247


In [ ]:
PROVIDER = 'Renner'
PEER_GROUP = 'SHOPPING'

market_dataframes = {}
ingestion_timestamp = datetime.now()

for market in market_reviews:
    reviews = market_reviews[market]
    if len(reviews) == 0: continue
    reviews_dict = [
        {
            'app_id': APP_ID,
            'country': market,
            'title': review['title'],
            'content': review['text'],
            'score': review['rating'],
            'review_date': review['review_date'],
            'fetched_at': ingestion_timestamp,
            'provider': PROVIDER,
            'peer_group': PEER_GROUP
        } for review in reviews
    ]
    df = pd.DataFrame(reviews_dict)
    market_dataframes[market] = df

In [8]:
for market in market_dataframes:
    df = market_dataframes[market]
    df.to_csv(f'{APP_ID}.{market}.csv', index=False)